数据预处理，注意标签为1或-1

In [22]:
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
import numpy as np
import random

def img2vector(directory, filename):
    vector1 = np.zeros((1, 1024))
    vector2 = np.zeros((1, 256))
    data = np.zeros((32, 32))
    f = open(directory + '/' + filename)
    for i in range(32):
        line = f.readline()
        for j in range(32):
            data[i, j] = int(line[j])
            vector1[0, 32*i+j] = int(line[j])
    for i in range(0, 32, 2):
        for j in range(0, 32, 2):
            vector2[0, int(i*8+j/2)] = data[i][j]+data[i][j+1]+data[i+1][j]+data[i+1][j+1]
    return vector1, vector2

def getData(filename):
    fileList = listdir(filename)
    m = len(fileList)
    dataset = np.zeros((m, 1024))
    dataset_DR = np.zeros((m, 256))
    labels = []
    for i in range(m):
        label = int(fileList[i].split('_')[0])
        labels.append(1 if label == 1 else -1)
        dataset[i], dataset_DR[i] = img2vector(filename, fileList[i])
    return np.mat(dataset), np.mat(dataset_DR), np.mat(labels).transpose()

# X_train, X_train_DR, y_train = getData('trainingDigits')
# X_test, X_test_DR, y_test = getData('testDigits')

保存计算需要用到的数据

In [23]:
# 计算核函数
def kernelTrans(X, A, kTup):
    m, _ = np.shape(X)
    K = np.mat(np.zeros((m,1)))
    if kTup[0] == 'lin':
        K = X * A.T                                        
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow*deltaRow.T
        K = np.exp(K/(-1*kTup[1]**2))
    else: raise NameError('核函数无法识别')
    return K
# 输入依次为数据集，标签集，松弛变量，容错率，核函数参数
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn                                              #数据矩阵
        self.labelMat = classLabels                                     #数据标签
        self.C = C                                                      #松弛变量
        self.tol = toler                                                #容错率
        self.m = np.shape(dataMatIn)[0]                                 #数据矩阵行数
        self.alphas = np.mat(np.zeros((self.m, 1)))                     #根据矩阵行数初始化alpha参数为0   
        self.b = 0                                                      #初始化b参数为0
        self.eCache = np.mat(np.zeros((self.m, 2)))                     #根据矩阵行数初始化虎误差缓存，第一列为是否有效的标志位，第二列为实际的误差E的值。
        self.K = np.mat(np.zeros((self.m, self.m)))                     #初始化核K
        for i in range(self.m):                                         #计算所有数据的核K
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)

def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T*oS.K[:,k] + oS.b)  #在这边和线性SVM有所区别，提前计算SVM核
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJrand(i, m):
    j = i
    while (j == i):
        j = int(random.uniform(0, m))
    return j

def selectJ(i, oS, Ei):
    maxK = -1; maxDeltaE = 0; Ej = 0                         #初始化
    oS.eCache[i] = [1, Ei]                                   #根据Ei更新误差缓存
    validEcacheList = np.nonzero(oS.eCache[:, 0].A)[0]       #返回误差不为0的数据的索引值
    if (len(validEcacheList)) > 1:                           #有不为0的误差
        for k in validEcacheList:                            #遍历,找到最大的Ek
            if k == i: continue                              #不计算i,浪费时间
            Ek = calcEk(oS, k)                               #计算Ek
            deltaE = abs(Ei - Ek)                            #计算|Ei-Ek|
            if (deltaE > maxDeltaE):                         #找到maxDeltaE
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej                                      #返回maxK,Ej
    else:                                                    #没有不为0的误差
        j = selectJrand(i, oS.m)                             #随机选择alpha_j的索引值
        Ej = calcEk(oS, j)                                   #计算Ej
    return j, Ej                                             #j,Ej

def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]

def clipAlpha(aj,H,L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

def innerL(i, oS):
    Ei = calcEk(oS, i)
    #选择违反KKT条件的ai，前者表示那些满足硬间隔的点但ai不等于C，后者表示符合硬间隔的点但ai不等于0
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            print("L==H", H)
            return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j]
        if eta >= 0:
            print("eta>=0", eta)
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.0001):
            print("alpha_j变化太小")
            return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])
        updateEk(oS, i)
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else:
        return 0

def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    oS = optStruct(dataMatIn, classLabels, C, toler, kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:                                                                                 #遍历整个数据集                           
            for i in range(oS.m):       
                alphaPairsChanged += innerL(i, oS)
                print("全样本遍历:第%d次迭代 样本:%d, alpha优化次数:%d" % (iter, i, alphaPairsChanged))
            iter += 1
        else:                                                                                         #遍历非边界值
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print("非边界遍历:第%d次迭代 样本:%d, alpha优化次数:%d" % (iter, i, alphaPairsChanged))
            iter += 1
        if entireSet:
            entireSet = False
        elif (alphaPairsChanged == 0):
            entireSet = True 
        print("迭代次数: %d" % iter)
    return oS.b, oS.alphas

if __name__ == '__main__':
    #数据预处理
    datMat, _, labelMat = getData('trainingDigits')
    #数据训练 
    b, alphas = smoP(datMat, labelMat, 200, 0.001, 10, ('rbf', 1.3))
    #只选取支持向量计算超平面
    svInd = np.nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]                                                     
    labelSV = labelMat[svInd]
    print("支持向量个数:%d" % np.shape(sVs)[0])
    m, _ = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i,:], ('rbf', 1.3))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelMat[i]): errorCount += 1
    print("训练集错误率: %.2f%%" % ((float(errorCount)/m)*100))
    datMat, _, labelMat = getData('testDigits')
    errorCount = 0     
    m, _ = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', 1.3))        
        predict=kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelMat[i]): errorCount += 1
    print("测试集错误率: %.2f%%" % ((float(errorCount)/m)*100))

L==H [[0.]]
全样本遍历:第0次迭代 样本:0, alpha优化次数:0
L==H [[0.]]
全样本遍历:第0次迭代 样本:1, alpha优化次数:0
全样本遍历:第0次迭代 样本:2, alpha优化次数:1
全样本遍历:第0次迭代 样本:3, alpha优化次数:2
全样本遍历:第0次迭代 样本:4, alpha优化次数:3
全样本遍历:第0次迭代 样本:5, alpha优化次数:4
全样本遍历:第0次迭代 样本:6, alpha优化次数:5
全样本遍历:第0次迭代 样本:7, alpha优化次数:6
全样本遍历:第0次迭代 样本:8, alpha优化次数:7
全样本遍历:第0次迭代 样本:9, alpha优化次数:8
全样本遍历:第0次迭代 样本:10, alpha优化次数:9
全样本遍历:第0次迭代 样本:11, alpha优化次数:10
全样本遍历:第0次迭代 样本:12, alpha优化次数:11
全样本遍历:第0次迭代 样本:13, alpha优化次数:12
全样本遍历:第0次迭代 样本:14, alpha优化次数:13
全样本遍历:第0次迭代 样本:15, alpha优化次数:14
全样本遍历:第0次迭代 样本:16, alpha优化次数:15
全样本遍历:第0次迭代 样本:17, alpha优化次数:16
全样本遍历:第0次迭代 样本:18, alpha优化次数:17
全样本遍历:第0次迭代 样本:19, alpha优化次数:18
全样本遍历:第0次迭代 样本:20, alpha优化次数:19
全样本遍历:第0次迭代 样本:21, alpha优化次数:20
全样本遍历:第0次迭代 样本:22, alpha优化次数:21
全样本遍历:第0次迭代 样本:23, alpha优化次数:22
全样本遍历:第0次迭代 样本:24, alpha优化次数:23
全样本遍历:第0次迭代 样本:25, alpha优化次数:24
全样本遍历:第0次迭代 样本:26, alpha优化次数:25
全样本遍历:第0次迭代 样本:27, alpha优化次数:26
全样本遍历:第0次迭代 样本:28, alpha优化次数:27
全样本遍历:第0次迭代 样本:29, alpha优化次数:28
全样本遍历:第0次迭代 样本:30, alpha优化次数:29
全样本遍历

In [24]:
import numpy as np
from os import listdir

def img2vector(directory, filename):
    vector1 = np.zeros((1, 1024))
    vector2 = np.zeros((1, 256))
    data = np.zeros((32, 32))
    f = open(directory + '/' + filename)
    for i in range(32):
        line = f.readline()
        for j in range(32):
            data[i, j] = int(line[j])
            vector1[0, 32*i+j] = int(line[j])
    for i in range(0, 32, 2):
        for j in range(0, 32, 2):
            vector2[0, int(i*8+j/2)] = data[i][j]+data[i][j+1]+data[i+1][j]+data[i+1][j+1]
    return vector1, vector2

def getData(filename):
    fileList = listdir(filename)
    m = len(fileList)
    dataset = np.zeros((m, 1024))
    dataset_DR = np.zeros((m, 256))
    labels = []
    for i in range(m):
        labels.append(int(fileList[i].split('_')[0]))
        dataset[i], dataset_DR[i] = img2vector(filename, fileList[i])
    return dataset, dataset_DR, labels

X_train, X_train_DR, y_train = getData('trainingDigits')
X_test, X_test_DR, y_test = getData('testDigits')

In [31]:
from sklearn.svm import SVC

if __name__ == '__main__':
    clf = SVC(C=100, kernel='rbf')
    clf.fit(X_train, y_train)
    result = clf.score(X_test, y_test)
    print(result)

0.9904862579281184
